In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from twilio.rest import Client
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:\Users\andre\Documents\GitHub\AI_lunch_box_recommender_system\.venv\.env")

# Create OpenAI and twilio bots 
client  = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
client_twilio = Client(os.environ.get('account_sid'), os.environ.get('auth_token') )

# Setting up the driver

url = "https://www.ica.se/erbjudanden/ica-supermarket-karlsborg-1003570/"
def scrape_you_local_ica(url):    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url) 

    # Cling button to agree cookies
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver,3)
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector)
    element.click()

    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Generate a soup
    soup = BeautifulSoup(driver.page_source)

    # Key to make a container to iterate over
    key = "article > div.offer-card__details-container"
    container = soup.select(key)

    # clean the output
    offers  = [i.text.replace("\xa0kr. ","") for i in container]
    offers = [i.split(". ")[:2] for i in offers]
    return offers

def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": prompt},
        ]
    )
    return completion.choices[0].message.content


def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": prompt},
        ]
    )
    return completion.choices[0].message.content

def send_text(from_number, to_number, text_message):
    message = client_twilio.messages.create(body=text_message, from_=from_number, to=to_number) 

# Prompt instructions for gpt bot
role = "Du är en kreativ kock som är väldigt duktig på att rekommendera enkla maträtter baserat på tillgängliga ingredienser."
instructions = "Jag vill att du rekommenderar mig maträtter som jag kan laga till middag och sedan göra matlåda av. Du får en lista med veckans rabatterade priser från en matbutik. Du ska använda listan för att skapa 3 enkla recept. Du får också lägga till basvaror (pasta, ris, potatis, lök etc.), även om de inte är med på listan.\nJag vill att du genererar 3 rätter i rubrikform och under varje rubrik/rätt så ska det finnas en lista med ingredienser för 4 portioner. Det är viktigt att enheterna är i enlighet med det metriska systemet. Under listan med ingredienser ska du beskriva väldigt kort hur man tillagar rätten."


#Scrape ICAs website
context = scrape_you_local_ica(url)
completion = chat_gpt_bot(client, role, instructions+str(context))
shopping_lst = chat_gpt_bot(client, "Hjälpreda", "Generera en inköpslista i punktform med sammanlagda enheter av följande information: "+completion)
#send_text(os.environ["my_twilio_number"], os.environ["my_phone_number"], shopping_lst)


# Write to file
with open("weekly_recipies.txt", "w") as file:
    file.write(completion)
    file.close()

# Write to shopping list to file (if twilio does not work)
with open("shopping_lst.txt", "w") as file:
    file.write(completion)
    file.close()




True